## 用Mnist数据集进行分类算法

In [1]:
# 防止内核挂掉
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

In [2]:
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

导入数据集

In [3]:
# 训练集
train_dataset = datasets.MNIST(root='./data/', 
                               train=True, 
                               transform=transforms.ToTensor(), 
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data/', 
                               train=False, 
                               transform=transforms.ToTensor(), 
                               download=True)

批次设置

In [4]:
# 批次大小
batch_size = 64 # 每次传入数据的量

# 装载训练集
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True) # 打乱数据集

# 装载测试集
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

查看第一个loader中的数据

In [5]:
inputs, labels = next(iter(train_loader))
print('input shape: ', inputs.shape)
print('label shape: ', labels.shape)
print('labels: ', labels)

input shape:  torch.Size([64, 1, 28, 28])
label shape:  torch.Size([64])
labels:  tensor([2, 5, 4, 5, 8, 2, 0, 1, 6, 2, 0, 0, 3, 4, 6, 4, 9, 7, 0, 1, 9, 1, 7, 3,
        0, 2, 0, 5, 9, 1, 4, 7, 4, 2, 9, 0, 0, 6, 6, 6, 2, 5, 1, 2, 6, 7, 7, 3,
        4, 5, 8, 6, 4, 2, 2, 3, 2, 7, 2, 0, 6, 6, 8, 1])


### 网络结构

In [6]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = nn.Sequential(nn.Linear(784, 500), nn.Dropout(p=0), nn.Tanh())
        self.layer2 = nn.Sequential(nn.Linear(500, 300), nn.Dropout(p=0), nn.Tanh())
        self.layer3 = nn.Sequential(nn.Linear(300, 10), nn.Softmax(dim=1))
        
    def forward(self, x):
        # 首先需要给数据进行处理，将原来[64, 1, 28, 28](样本数，通道数，长，宽)变为[64, 728]的形状
        # 全连接层输入必须是二维数据
        x = x.view(x.size()[0], -1)
        
        z1 = self.layer1(x)
        z2 = self.layer2(z1)
        output = self.layer3(z2)
        
        return output

### 模型参数

In [7]:
# 定义模型
model = Net()

# 损失函数
crossentropy_loss = nn.CrossEntropyLoss()

# 优化器，设置L2正则化
optimizer = optim.SGD(model.parameters(), lr=0.5, weight_decay=0.001)

### 训练与测试的方法

In [8]:
# 训练的方法
def train():
    # 模型的训练状态
    # 模型会Dropout
    model.train()
    for i, data in enumerate(train_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 计算loss
        loss = crossentropy_loss(out, labels) # 交叉熵不用one-hot编码
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

In [9]:
# 测试的方法
def test():
    # 模型的验证测试状态
    # 模型不会Dropout
    model.eval()
    correct = 0
    for i, data in enumerate(test_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 获得最大值与其所在位置
        _, predicted = torch.max(out, 1)
        # 预测正确的个数
        correct += (predicted == labels).sum()
    
    print(f'Test acc: {correct.item()/len(test_dataset)}')

    correct = 0
    for i, data in enumerate(train_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 获得最大值与其所在位置
        _, predicted = torch.max(out, 1)
        # 预测正确的个数
        correct += (predicted == labels).sum()
    
    print(f'Train acc: {correct.item()/len(train_dataset)}')

In [10]:
for epoch in range(20):
    print(f'epoch: {epoch}')
    train()
    test()

epoch: 0
Test acc: 0.9105
Train acc: 0.9081666666666667
epoch: 1
Test acc: 0.9102
Train acc: 0.9102166666666667
epoch: 2
Test acc: 0.9258
Train acc: 0.9252833333333333
epoch: 3
Test acc: 0.9349
Train acc: 0.9358
epoch: 4
Test acc: 0.9395
Train acc: 0.9405666666666667
epoch: 5
Test acc: 0.939
Train acc: 0.93995
epoch: 6
Test acc: 0.9457
Train acc: 0.9474666666666667
epoch: 7
Test acc: 0.9466
Train acc: 0.9467666666666666
epoch: 8
Test acc: 0.9495
Train acc: 0.9521333333333334
epoch: 9
Test acc: 0.9476
Train acc: 0.9514166666666667
epoch: 10
Test acc: 0.9457
Train acc: 0.94935
epoch: 11
Test acc: 0.9537
Train acc: 0.95695
epoch: 12
Test acc: 0.9482
Train acc: 0.9512833333333334
epoch: 13
Test acc: 0.9496
Train acc: 0.9514833333333333
epoch: 14
Test acc: 0.9557
Train acc: 0.9588166666666667
epoch: 15
Test acc: 0.9569
Train acc: 0.9588833333333333
epoch: 16
Test acc: 0.9603
Train acc: 0.9623833333333334
epoch: 17
Test acc: 0.9443
Train acc: 0.9464666666666667
epoch: 18
Test acc: 0.9591
Tra